In [1]:
import hashlib
import numpy as np
import pandas as pd

from datasetsforecast.m4 import M4

In [194]:
def _create_ds(unique_id, start, periods, freq):
    try:
        total_dates = pd.date_range(start=start, periods=periods, freq=freq)
        time_series = pd.DataFrame({'ts_name': unique_id, 'ds': total_dates})
    except:
        print('unique-id: ', unique_id)
        print('start: ', start)
        print('periods: ', periods)
        print('freq: ', freq)
    return time_series

## MONTHLY

In [3]:
def parse_monthly(Y_df, dates):

    counts = Y_df.groupby('unique_id').count().reset_index()[['unique_id', 'y']]
    counts.columns = ['unique_id', 'count']

    dates = dates.copy()
    dates = dates[dates['SP'] == 'Monthly'].reset_index(drop=True)
    dates = dates.merge(counts, left_on='M4id', right_on='unique_id')

    def parse_date(x):
        if len(x) == 14:
            if int(x[6:8]) < 17:
                return '20' + x[6:8] + '-' + x[3:5] + '-01'
            else:
                return '19' + x[6:8] + '-' + x[3:5] + '-01'
        elif len(x) == 19:
            return x[:10]

    dates['StartingDate'] = dates['StartingDate'].apply(parse_date)
    assert len(dates[dates['StartingDate'].isnull()]) == 0

    ds_df = pd.concat([_create_ds(row[1]['M4id'],
                                row[1]['StartingDate'],
                                row[1]['count'],
                                frequency_map['Monthly']) for row in dates.iterrows()])

    ds_df = ds_df.sort_values(by=['ts_name', 'ds']).reset_index(drop=True)
    np.all(Y_df['unique_id'] == ds_df['ts_name'])
    Y_df['ds'] = ds_df['ds']

    return Y_df

In [4]:
frequency_map = {'Monthly': 'MS', 'Quarterly': 'Q', 'Yearly': 'YS', 'Weekly': 'W'}
Y_df, _, _ = M4.load(directory='./', group='Monthly', cache=True)
dates = pd.read_csv('https://raw.githubusercontent.com/Mcompetitions/M4-methods/master/Dataset/M4-info.csv')

In [5]:
Y_df = parse_monthly(Y_df, dates)

In [6]:
Y_df['dataset'] = 'M4_monthly'
Y_df['frequency'] = 'MS'
Y_df['ts_name'] = Y_df['unique_id']
Y_df['to_hash'] = Y_df['dataset'] + '_' + Y_df['unique_id']
Y_df['unique_id'] = Y_df['to_hash'].apply(lambda x: hashlib.sha1(x.encode()).hexdigest())
Y_df = Y_df[['unique_id', 'ds', 'y', 'dataset', 'ts_name','frequency']]

In [7]:
Y_df.to_parquet('M4_monthly.parquet', index=False)

# YEARLY

In [8]:
def parse_yearly(Y_df, dates):

    counts = Y_df.groupby('unique_id').count().reset_index()[['unique_id', 'y']]
    counts.columns = ['unique_id', 'counts']

    # Filter older series
    to_drop = counts[counts['counts']>200].unique_id.values
    Y_df = Y_df[~Y_df['unique_id'].isin(to_drop)].reset_index(drop=True)
    counts = counts[~counts['unique_id'].isin(to_drop)].reset_index(drop=True)

    dates = dates.copy()
    dates = dates[dates['SP'] == 'Yearly'].reset_index(drop=True)
    dates = dates.merge(counts, left_on='M4id', right_on='unique_id')

    def parse_date(ds, count):
        if len(ds) == 14:
            year = int(ds[6:8])
            if year+2000+count <= 2017:
                return '20' + ds[6:8] + '-01-01'
            elif year+1900+count <= 2017:
                return '19' + ds[6:8] + '-01-01'
            elif year+1800+count <= 2017:
                return '18' + ds[6:8] + '-01-01'
            elif year+1700+count <= 2017:
                return '17' + ds[6:8] + '-01-01'
            elif year+1600+count <= 2017:
                return '16' + ds[6:8] + '-01-01'
            else:
                print(year)
                print(count)
                raise Exception('Year is too old')
        elif len(ds) == 19:
            return ds[:10]

    dates['StartingDate'] = dates.apply(lambda x: parse_date(x.StartingDate, x.counts), axis=1)

    assert len(dates[dates['StartingDate'].isnull()]) == 0

    ds_df = pd.concat([_create_ds(row[1]['M4id'],
                                row[1]['StartingDate'],
                                row[1]['counts'],
                                frequency_map['Yearly']) for row in dates.iterrows()])

    ds_df = ds_df.sort_values(by=['ts_name', 'ds']).reset_index(drop=True)
    np.all(Y_df['unique_id'] == ds_df['ts_name'])
    Y_df['ds'] = ds_df['ds']

    return Y_df

In [9]:
frequency_map = {'Monthly': 'MS', 'Quarterly': 'Q', 'Yearly': 'YS', 'Weekly': 'W'}
Y_df, _, _ = M4.load(directory='./', group='Yearly', cache=True)
dates = pd.read_csv('https://raw.githubusercontent.com/Mcompetitions/M4-methods/master/Dataset/M4-info.csv')

In [10]:
Y_df = parse_yearly(Y_df, dates)

In [11]:
Y_df['dataset'] = 'M4_yearly'
Y_df['frequency'] = 'YS'
Y_df['ts_name'] = Y_df['unique_id']
Y_df['to_hash'] = Y_df['dataset'] + '_' + Y_df['unique_id']
Y_df['unique_id'] = Y_df['to_hash'].apply(lambda x: hashlib.sha1(x.encode()).hexdigest())
Y_df = Y_df[['unique_id', 'ds', 'y', 'dataset', 'ts_name','frequency']]

In [12]:
Y_df.to_parquet('M4_yearly.parquet', index=False)

# Quarterly

In [104]:
def parse_quarterly(Y_df, dates):

    counts = Y_df.groupby('unique_id').count().reset_index()[['unique_id', 'y']]
    counts.columns = ['unique_id', 'counts']

    # Filter older series
    to_drop = counts[counts['counts']>800].unique_id.values
    Y_df = Y_df[~Y_df['unique_id'].isin(to_drop)].reset_index(drop=True)
    counts = counts[~counts['unique_id'].isin(to_drop)].reset_index(drop=True)

    dates = dates.copy()
    dates = dates[dates['SP'] == 'Quarterly'].reset_index(drop=True)
    dates = dates.merge(counts, left_on='M4id', right_on='unique_id')

    def parse_date(ds, count):
        if len(ds) == 14:
            year = int(ds[6:8])
            if year+2000+count//4 <= 2017:
                return '20' + ds[6:8] + '-' + ds[3:5] + '-01'
            elif year+1900+count//4 <= 2017:
                return '19' + ds[6:8] + '-' + ds[3:5] + '-01'
            elif year+1800+count//4 <= 2017:
                return '18' + ds[6:8] + '-' + ds[3:5] + '-01'
            elif year+1700+count//4 <= 2017:
                return '17' + ds[6:8] + '-' + ds[3:5] + '-01'
            elif year+1600+count//4 <= 2017:
                return '16' + ds[6:8] + '-' + ds[3:5] + '-01'
            else:
                print(year)
                print(count)
                raise Exception('Year is too old')
        elif len(ds) == 19:
            return ds[:10]

    dates['StartingDate'] = dates.apply(lambda x: parse_date(x.StartingDate, x.counts), axis=1)

    assert len(dates[dates['StartingDate'].isnull()]) == 0

    ds_df = pd.concat([_create_ds(row[1]['M4id'],
                                row[1]['StartingDate'],
                                row[1]['counts'],
                                frequency_map['Quarterly']) for row in dates.iterrows()])

    ds_df = ds_df.sort_values(by=['ts_name', 'ds']).reset_index(drop=True)
    np.all(Y_df['unique_id'] == ds_df['ts_name'])
    Y_df['ds'] = ds_df['ds']

    return Y_df

In [105]:
dates = dates[dates['SP'] == 'Quarterly'].reset_index(drop=True)
dates

,M4id,category,Frequency,Horizon,SP,StartingDate
0,Q1,Macro,4,8,Quarterly,01-01-05 12:00
1,Q2,Macro,4,8,Quarterly,01-01-05 12:00
2,Q3,Macro,4,8,Quarterly,01-01-05 12:00
3,Q4,Macro,4,8,Quarterly,01-01-05 12:00
4,Q5,Macro,4,8,Quarterly,01-01-00 12:00
...,...,...,...,...,...,...
23995,Q23996,Other,4,8,Quarterly,31-03-97 12:00
23996,Q23997,Other,4,8,Quarterly,31-03-97 12:00
23997,Q23998,Other,4,8,Quarterly,31-03-97 12:00
23998,Q23999,Other,4,8,Quarterly,30-06-99 12:00


In [106]:
frequency_map = {'Monthly': 'MS', 'Quarterly': 'QS', 'Yearly': 'YS', 'Weekly': 'W'}
Y_df, _, _ = M4.load(directory='./', group='Quarterly', cache=True)
dates = pd.read_csv('https://raw.githubusercontent.com/Mcompetitions/M4-methods/master/Dataset/M4-info.csv')

In [107]:
Y_df = parse_quarterly(Y_df, dates)

In [108]:
Y_df['dataset'] = 'M4_quarterly'
Y_df['frequency'] = 'QS'
Y_df['ts_name'] = Y_df['unique_id']
Y_df['to_hash'] = Y_df['dataset'] + '_' + Y_df['unique_id']
Y_df['unique_id'] = Y_df['to_hash'].apply(lambda x: hashlib.sha1(x.encode()).hexdigest())
Y_df = Y_df[['unique_id', 'ds', 'y', 'dataset', 'ts_name','frequency']]

In [114]:
Y_df.to_parquet('M4_quarterly.parquet', index=False)

## Daily

In [153]:
def parse_daily(Y_df, dates):

    counts = Y_df.groupby('unique_id').count().reset_index()[['unique_id', 'y']]
    counts.columns = ['unique_id', 'counts']

    dates = dates.copy()
    dates = dates[dates['SP'] == 'Daily'].reset_index(drop=True)
    dates = dates.merge(counts, left_on='M4id', right_on='unique_id')

    def parse_date(ds, count):
        if len(ds) == 14:
            year = int(ds[6:8])
            if year+2000+count//365 < 2017:
                return '20' + ds[6:8] + ds[3:5] + ds[0:2]
            elif year+1900+count//365 < 2017:
                return '19' + ds[6:8] + ds[3:5] + ds[0:2]
            elif year+1800+count//365 < 2017:
                return '18' + ds[6:8] + ds[3:5] + ds[0:2]
            elif year+1700+count//365 < 2017:
                return '17' + ds[6:8] + ds[3:5] + ds[0:2]
            elif year+1600+count//365 < 2017:
                return '16' + ds[6:8] + ds[3:5] + ds[0:2]
            else:
                print(year)
                print(count)
                raise Exception('Year is too old')
        elif len(ds) == 19:
            return ds[:10]

    dates['StartingDate'] = dates.apply(lambda x: parse_date(x.StartingDate, x.counts), axis=1)

    assert len(dates[dates['StartingDate'].isnull()]) == 0

    ds_df = pd.concat([_create_ds(row[1]['M4id'],
                                row[1]['StartingDate'],
                                row[1]['counts'],
                                frequency_map['Daily']) for row in dates.iterrows()])

    ds_df = ds_df.sort_values(by=['ts_name', 'ds']).reset_index(drop=True)
    np.all(Y_df['unique_id'] == ds_df['ts_name'])
    Y_df['ds'] = ds_df['ds']

    return Y_df

In [154]:
frequency_map = {'Monthly': 'MS', 'Quarterly': 'QS', 'Yearly': 'YS', 'Weekly': 'W', 'Daily':'D'}
Y_df, _, _ = M4.load(directory='./', group='Daily', cache=True)
dates = pd.read_csv('https://raw.githubusercontent.com/Mcompetitions/M4-methods/master/Dataset/M4-info.csv')

In [156]:
Y_df = parse_daily(Y_df, dates)

In [157]:
Y_df['dataset'] = 'M4_daily'
Y_df['frequency'] = 'D'
Y_df['ts_name'] = Y_df['unique_id']
Y_df['to_hash'] = Y_df['dataset'] + '_' + Y_df['unique_id']
Y_df['unique_id'] = Y_df['to_hash'].apply(lambda x: hashlib.sha1(x.encode()).hexdigest())
Y_df = Y_df[['unique_id', 'ds', 'y', 'dataset', 'ts_name','frequency']]

In [158]:
Y_df.to_parquet('M4_daily.parquet', index=False)

In [159]:
Y_df

,unique_id,ds,y,dataset,ts_name,frequency
0,41e523f5ff6069aa70b7ea7d081a315166b00d81,1994-01-03,1017.10,M4_daily,D1,D
1,41e523f5ff6069aa70b7ea7d081a315166b00d81,1994-01-04,1019.30,M4_daily,D1,D
2,41e523f5ff6069aa70b7ea7d081a315166b00d81,1994-01-05,1017.00,M4_daily,D1,D
3,41e523f5ff6069aa70b7ea7d081a315166b00d81,1994-01-06,1019.20,M4_daily,D1,D
4,41e523f5ff6069aa70b7ea7d081a315166b00d81,1994-01-07,1018.70,M4_daily,D1,D
...,...,...,...,...,...,...
10023831,7b0081dd78259bdd3b9d5c46ad3e2ba18922f0a6,2016-08-16,1262.08,M4_daily,D999,D
10023832,7b0081dd78259bdd3b9d5c46ad3e2ba18922f0a6,2016-08-17,1262.78,M4_daily,D999,D
10023833,7b0081dd78259bdd3b9d5c46ad3e2ba18922f0a6,2016-08-18,1264.66,M4_daily,D999,D
10023834,7b0081dd78259bdd3b9d5c46ad3e2ba18922f0a6,2016-08-19,1272.66,M4_daily,D999,D


# Hourly

In [226]:
def parse_hourly(Y_df, dates):

    counts = Y_df.groupby('unique_id').count().reset_index()[['unique_id', 'y']]
    counts.columns = ['unique_id', 'counts']

    dates = dates.copy()
    dates = dates[dates['SP'] == 'Hourly'].reset_index(drop=True)
    dates = dates.merge(counts, left_on='M4id', right_on='unique_id')

    def parse_date(ds, count):
        if len(ds) == 14:
            year = int(ds[6:8])
            if year+2000+count//(365*24) < 2018:
                return '20' + ds[6:8] + '-' + ds[3:5] + '-' + ds[0:2] + ' ' + ds[9:11] + ':00:00'
            elif year+1900+count//(365*24) < 2018:
                return '19' + ds[6:8] + '-' + ds[3:5] + '-' + ds[0:2] + ' ' + ds[9:11] + ':00:00'
            elif year+1800+count//(365*24) < 2018:
                return '18' + ds[6:8] + '-' + ds[3:5] + '-' + ds[0:2] + ' ' + ds[9:11] + ':00:00'
        if len(ds) == 13:
            year = int(ds[6:8])
            if year+2000+count//(365*24) < 2018:
                return '20' + ds[6:8] + '-' + ds[3:5] + '-' + ds[0:2] + ' ' + ds[9:10] + ':00:00'
            elif year+1900+count//(365*24) < 2018:
                return '19' + ds[6:8] + '-' + ds[3:5] + '-' + ds[0:2] + ' ' + ds[9:10] + ':00:00'
            elif year+1800+count//(365*24) < 2018:
                return '18' + ds[6:8] + '-' + ds[3:5] + '-' + ds[0:2] + ' ' + ds[9:10] + ':00:00'

    dates['StartingDate'] = dates.apply(lambda x: parse_date(x.StartingDate, x.counts), axis=1)

    assert len(dates[dates['StartingDate'].isnull()]) == 0

    ds_df = pd.concat([_create_ds(row[1]['M4id'],
                                row[1]['StartingDate'],
                                row[1]['counts'],
                                frequency_map['Hourly']) for row in dates.iterrows()])

    ds_df = ds_df.sort_values(by=['ts_name', 'ds']).reset_index(drop=True)
    np.all(Y_df['unique_id'] == ds_df['ts_name'])
    Y_df['ds'] = ds_df['ds']

    return Y_df

In [227]:
frequency_map = {'Monthly': 'MS', 'Quarterly': 'QS', 'Yearly': 'YS', 'Weekly': 'W', 'Daily':'D', 'Hourly':'H'}
Y_df, _, _ = M4.load(directory='./', group='Hourly', cache=True)
dates = pd.read_csv('https://raw.githubusercontent.com/Mcompetitions/M4-methods/master/Dataset/M4-info.csv')

In [222]:
Y_df = parse_hourly(Y_df, dates)

In [223]:
Y_df['dataset'] = 'M4_hourly'
Y_df['frequency'] = 'H'
Y_df['ts_name'] = Y_df['unique_id']
Y_df['to_hash'] = Y_df['dataset'] + '_' + Y_df['unique_id']
Y_df['unique_id'] = Y_df['to_hash'].apply(lambda x: hashlib.sha1(x.encode()).hexdigest())
Y_df = Y_df[['unique_id', 'ds', 'y', 'dataset', 'ts_name','frequency']]

In [224]:
Y_df.to_parquet('M4_hourly.parquet', index=False)